# **BA 870 – Assignment 4**

**Shangkun(Sherry) Zuo, Yuqi(Yoki) Liu, Yanni Lan, Jiayuan Zou, Ziyan Pei, Siqi Zhang**  
Cohort B  
April 13, 2020

## Import Data

### Upload data `factors` to Pandas Dataframes

In [0]:
from google.colab import files
uploaded = files.upload()

Saving factors.csv to factors.csv


In [0]:
import pandas as pd
import numpy as np
df = pd.read_csv('factors.csv')
pd.DataFrame.from_records(df)
df.head()

,Date (SAS). Last Trading Day of the Month,Excess Return on the Market,Small-Minus-Big Return,High-Minus-Low Return,Risk-Free Return Rate (One Month Treasury Bill Rate)
0,20160129,-0.0577,-0.0335,0.0208,0.0001
1,20160229,-0.0007,0.0079,-0.0050,0.0002
2,20160331,0.0696,0.0087,0.0116,0.0002
3,20160429,0.0092,0.0069,0.0326,0.0001
4,20160531,0.0178,-0.0027,-0.0181,0.0001


In [0]:
# Rename Columns
renamed_columns = ['Date', "Rm_minus_Rf", "SMB", "HML", "Rf"]
df.columns = renamed_columns
df.head()

,Date,Rm_minus_Rf,SMB,HML,Rf
0,20160129,-0.0577,-0.0335,0.0208,0.0001
1,20160229,-0.0007,0.0079,-0.0050,0.0002
2,20160331,0.0696,0.0087,0.0116,0.0002
3,20160429,0.0092,0.0069,0.0326,0.0001
4,20160531,0.0178,-0.0027,-0.0181,0.0001


### Upload data `returns` to Pandas Dataframes

In [0]:
from google.colab import files
uploaded = files.upload()

Saving returns.csv to returns.csv


In [0]:
import pandas as pd
import numpy as np
df2 = pd.read_csv('returns.csv')
pd.DataFrame.from_records(df2)
df2.head()

,PERMNO,Names Date,Ticker Symbol,Returns
0,10026,20160129,JJSF,-0.074484
1,10026,20160229,JJSF,0.026023
2,10026,20160331,JJSF,-0.019135
3,10026,20160429,JJSF,-0.066033
4,10026,20160531,JJSF,0.043212


In [0]:
# Rename Columns
renamed_columns = ['PERMNO', "Date", "Ticker", "Ri"]
df2.columns = renamed_columns
df2.head()

,PERMNO,Date,Ticker,Ri
0,10026,20160129,JJSF,-0.074484
1,10026,20160229,JJSF,0.026023
2,10026,20160331,JJSF,-0.019135
3,10026,20160429,JJSF,-0.066033
4,10026,20160531,JJSF,0.043212


## Data Cleaning

In [0]:
#check any NA in each variable in df
df.isna().any()

Date           False
Rm_minus_Rf    False
SMB            False
HML            False
Rf             False
dtype: bool

In [0]:
#check any NA in each variable in df2
df2.isna().any()

PERMNO    False
Date      False
Ticker     True
Ri         True
dtype: bool

In [0]:
#counting missing values in each variable in df2
df2.isna().sum()

PERMNO      0
Date        0
Ticker    273
Ri        200
dtype: int64

We investigate the rows with missing values for `Ticker` and `Ri` variables.

In [0]:
def nans(df2): 
  return df2[df2.isnull().any(axis=1)]

In [0]:
missing = nans(df2)
missing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 574 to 62133
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   PERMNO  303 non-null    int64 
 1   Date    303 non-null    int64 
 2   Ticker  30 non-null     object
 3   Ri      103 non-null    object
dtypes: int64(2), object(2)
memory usage: 11.8+ KB


Above, we can see that the total number of rows in df2 that have missing values is 303.

In [0]:
res = missing.loc[missing['Ticker'].isnull(), 'Ri'].tolist()
## check the values of Ri when Ticker is missing
print(set(res))
## check the values of Ticker when Ri is missing
res2 = df2.loc[df2['Ri'].isnull(),'Ticker'].tolist()
print(set(res2))

{nan, 'B'}
{nan, 'ADPT', 'INSY', 'CB', 'JCI', 'SMCI', 'GSHT', 'TW', 'DD', 'LILA', 'BZC', 'MWW', 'TFC', 'TIVO', 'JNP', 'TCF', 'ATAC', 'IQNT', 'CASM', 'LKCO', 'OHGI', 'TFCF', 'ORGO', 'RPTP', 'FTI', 'TFCFA', 'AHPA', 'DOW', 'LILAK', 'GDP'}


We also go back to observe the original `returns` dataset. For rows where `Ticker` is empty and for rows where `Ri` labeled as 'C', it's usually where the first few months of the stock, so that these stocks may not have any stock return, and we could not use the mean to substitute those missing values. 

Therefore, we remove those empty rows. Same reason for removal of rows with `Ri` labaled as 'B', since this situation always happen when `Ticker` is missing. 

In other words, when rows with `Ri` is labeled as 'B' or 'C' instead of numerical values, `Ri`s of these observations are missing even though these `Ri`s here are not recorded as absolute missing (NaN) by WRDS dataset. Therefore, we need to remove all rows whose `Ri` equals to 'B' or 'C'. 

In [0]:
#remove missing values
df2.dropna(inplace=True)

In [0]:
df2.drop(df2[df2['Ri'] == 'C'].index, inplace=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61741 entries, 0 to 62229
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   PERMNO  61741 non-null  int64 
 1   Date    61741 non-null  int64 
 2   Ticker  61741 non-null  object
 3   Ri      61741 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.4+ MB


In [0]:
df2.isna().sum()

PERMNO    0
Date      0
Ticker    0
Ri        0
dtype: int64

Check the number of unique `PERMNO` and `Tickers`.

In [0]:
len(df2['PERMNO'].unique().tolist())

1395

In [0]:
len(df2['Ticker'].unique().tolist())

1457

There are **1395** unique PERMNO and **1457** unique Ticker Symbols, so we need to check what happen.

In [0]:
#check each PERMNO Count
df2['PERMNO'].value_counts()

22517    48
92775    48
75694    48
34746    48
83906    48
         ..
18961     3
19031     2
18913     2
18968     2
19032     2
Name: PERMNO, Length: 1395, dtype: int64

The maximum `PERMNO` count is 48 which matches with `factors` dataset. Back to the original `returns` dataset, we found that one PERMNO corresponds to more than one Ticker Symbol but keeps the time change. The reason is that because `PERMNO` is a Permanent Identifier while `Ticker` is an identifier that tends to change over time. 

For example, if a company ceases to exist or bankrupt, its ticker may be reassigned to another company. For these reasons, `Ticker` doesn't work well with our historial anaylsis based upon date. So we want to keep the newest Ticker Symbol up to date.

In [0]:
pt=df2.sort_values('Date').groupby('PERMNO').tail(1)[['Ticker','PERMNO']]

In [0]:
pt['PERMNO'].astype(int)
pt_key=pt['PERMNO'].values.tolist()
pt_value=pt['Ticker'].tolist()
pt_dict = dict(zip(pt_key, pt_value))

In [0]:
#replace Ticker with newest Ticker Symbol when they have same PERMNO
df2['Ticker']= df2['PERMNO'].map(pt_dict)

In [0]:
#check Ticker again
len(df2['Ticker'].unique().tolist())

1380

After cleaning `Ticker` Symbol, we need to deal with the situation that why there are more PERMNOs than Ticker Symbols

In [0]:
#check each Ticker Count
df2['Ticker'].value_counts()

TAP     96
LEN     96
MKC     96
STZ     96
TCF     91
        ..
SWTX     3
PNTG     2
PGNY     2
VIR      2
VIE      2
Name: Ticker, Length: 1380, dtype: int64

In [0]:
#Regularly, each sticker should have 48 rows, so check what happens to TAP as an example since it has 96 counts
df2[df2['Ticker'] == 'TAP']['PERMNO'].value_counts()

59248    48
90562    48
Name: PERMNO, dtype: int64

In [0]:
#Check what happen to TCF
df2[df2['Ticker'] == 'TCF']['PERMNO'].value_counts()

11992    48
10375    43
Name: PERMNO, dtype: int64

In [0]:
#Check what happen to DDOG
df2[df2['Ticker'] == 'DDOG']['PERMNO'].value_counts()

18911    3
Name: PERMNO, dtype: int64

We drop those duplicate PERMNO rows with same `Date` and `Ticker` Symbol, and keep the first row if there is a duplication since the one with more rows appear first for TCF when we check back to the original dataset

In [0]:
df3 = df2.drop_duplicates(subset=['Date', 'Ticker'], keep='first')

In [0]:
df3['Ticker'].value_counts()

ARNA    48
EQR     48
PPL     48
RGLD    48
TRUP    48
        ..
SDC      3
VIE      2
VIR      2
PNTG     2
PGNY     2
Name: Ticker, Length: 1380, dtype: int64

Also, for Ticker symbols that less than 4 rows, they do not help us to analyze the stock performance, and their total data records are too less for regression of CAPM or Fama-French, so we want to keep `Ticker` Symbol at least for 6 months (half year) for our analysis since we do not want to remove too many rows but remove outliers.

In [0]:
Tsize=df3.groupby(['Ticker']).Ticker.size()
Tlist=Tsize[Tsize >= 6].index.tolist()

In [0]:
df4=df3[df3.Ticker.isin(Tlist)]

In [0]:
df4['Ticker'].value_counts()

ARNA    48
TRUP    48
PSX     48
C       48
HON     48
        ..
CHNG     6
AMCR     6
GO       6
BBIO     6
REAL     6
Name: Ticker, Length: 1364, dtype: int64

##Estimates the CAPM Beta

In [0]:
#Import "Statsmodels" library for regressions
import statsmodels.api as sm

>#### **CAPM**  
(Capital Asset Pricing Model)  

1\. Formula:   

$(Ri-Rf) = Alpha + Beta*(Rmkt-Rf) + e$ 

2\. Explanation:   

$Ri$: Stock Return   

$Rf$: Risk-Free Rate of Return  

$Rmkt(Rm)$: Market Return  

$Alpha$: Differences between the assets' return and the expected return, which measure the reward or penalty for holding idiosyncratic risk.  

$Beta$: Assets Systematic risk, which measure the sensitivity of the stock to the movement of the market portfolio. (market risk)  

$e$: Idiosyncratic Risk which the firm-specific risk. (unsystematic risk)

In [0]:
def CAPM(data, r):
  X = df[df.Date.isin(data['Date'])]['Rm_minus_Rf'].values
  y = (data[r].astype(float).values - df[df.Date.isin(data['Date'])]['Rf'].values).tolist()#df4['Ri']-df['Rf']
  X = sm.add_constant(X) # adding a constant
  model = sm.OLS(y, X).fit()
  return model.params[1]

In [0]:
capm=df4.groupby('Ticker').apply(CAPM, 'Ri')

## Estimate the Fama-French 3-Factor Betas

>#### **Fama-French Three-Factor Model**   

1\. Formula:   

$(Ri-Rf) = Alpha + Beta*(Rmkt-Rf) + s*SMB + h*HML + e$ 

2\. Explanation:   

$Ri$: Stock Return   

$Rf$: Risk-Free Rate of Return  

$Rmkt(Rm)$: Market Return  

$Alpha$: Differences between the assets' return and the expected return, which measure the reward or penalty for holding idiosyncratic risk.  

$Beta$: Assets Systematic risk, which measure the sensitivity of the stock to the movement of the market portfolio. (market risk) beta with respect to market   

$e$: Idiosyncratic Risk which the firm-specific risk. (unsystematic risk) 

$s$: beta with respect to size    

$h$: beta with respect to value    

$SMB$: Size Factor  

$HML$: Distress Factor or Value Factor

In [0]:
def Fama_French(data, r):
  X = df[df.Date.isin(data['Date'])][['Rm_minus_Rf', 'SMB', 'HML']].values
  y = (data[r].astype(float).values - df[df.Date.isin(data['Date'])]['Rf'].values).tolist()#df4['Ri']-df['Rf']
  X = sm.add_constant(X) # adding a constant
  model = sm.OLS(y, X).fit()
  return model.params[1:4]

In [0]:
fama_french=df4.groupby('Ticker').apply(Fama_French, 'Ri')

## Statistics Description

#### CAPM

In [0]:
capm

Ticker
A       1.249362
AAL     1.803727
AAON    0.950044
AAP     1.006860
AAPL    1.172580
          ...   
ZLAB    1.348280
ZM     -0.067142
ZNGA    0.504836
ZS      1.101106
ZTS     0.690075
Length: 1364, dtype: float64

In [0]:
capm=capm.to_frame(name="CAPM_Beta")

In [0]:
capm.head()

,CAPM_Beta
Ticker,
A,1.249362
AAL,1.803727
AAON,0.950044
AAP,1.006860
AAPL,1.172580


In [0]:
capm.describe()

,CAPM_Beta
count,1364.000000
mean,1.243769
std,1.341235
min,-5.007425
25%,0.767604
50%,1.169672
75%,1.545689
max,40.060448


**Summarize**:   
i). The average CAPM Beta across all stocks is 1.24  
ii). The minimum is -5.01, the 25th percentile is 0.77, the median is 1.17, the 75th percentile is 1.55, and the maximum is 40.06 for CAPM Beta. 

**There are some outliers since the maximum and minimum are far away the IQR boundary.**

In [0]:
from scipy.stats.mstats import winsorize
# Convert variables
# Using Winsorize to convert outliers
for col in capm.columns:
  capm[col] = winsorize(capm[col], limits=[0.05,0.07], inplace=True)
  #stats.winsorize(limits = [lower,upper])

In [0]:
capm.describe()

,CAPM_Beta
count,1364.000000
mean,1.182811
std,0.584877
min,0.164903
25%,0.767604
50%,1.169672
75%,1.545689
max,2.291931


**Summarize**:   
i). The average CAPM Beta across all stocks is 1.18  
ii). The minimum is 0.16, the 25th percentile is 0.77, the median is 1.17, the 75th percentile is 1.55, and the maximum is 2.29 for CAPM Beta. 

**The range is more reasonable now.**

#### Fama-French Three-Factor Model

In [0]:
fama_french

Ticker
A       [1.3978103196043847, -0.5402075201203762, -0.3...
AAL     [1.7988552604686967, -0.025964565501159498, 0....
AAON    [0.9289649934894557, 0.07824942391455628, 0.02...
AAP     [0.8759471515070184, 0.45354604370153095, 0.62...
AAPL    [1.296142814191379, -0.4161797823140478, -0.77...
                              ...                        
ZLAB    [1.1893338558144673, 0.7383777230608509, -1.55...
ZM      [0.3905902578117175, -0.21208033594696052, -1....
ZNGA    [0.15967543299671438, 1.369959842830642, -1.04...
ZS      [1.5958078973518302, -1.7286520105709569, -3.4...
ZTS     [0.6637505209787407, 0.12204729398811653, -0.3...
Length: 1364, dtype: object

In [0]:
fama_french = pd.DataFrame(fama_french.values.tolist(), columns=['FF-Beta','FF-Size','FF-Value'])

In [0]:
fama_french.head()

,FF-Beta,FF-Size,FF-Value
0,1.397810,-0.540208,-0.310123
1,1.798855,-0.025965,0.686844
2,0.928965,0.078249,0.020157
3,0.875947,0.453546,0.627367
4,1.296143,-0.416180,-0.776466


In [0]:
fama_french.describe()

,FF-Beta,FF-Size,FF-Value
count,1364.000000,1364.000000,1364.000000
mean,1.113419,0.575382,-0.034652
std,2.000123,1.962073,1.286439
min,-10.567507,-26.685662,-23.102998
25%,0.642078,-0.070789,-0.475491
50%,1.027332,0.394918,0.017082
75%,1.407668,1.026995,0.551794
max,63.573366,21.228339,10.341841


**Summarize**:     

>Fama-French Beta  

i). The average Fama-French Beta across all stocks is 1.11  
ii). The minimum is -10.57, the 25th percentile is 0.64, the median is 1.03, the 75th percentile is 1.41, and the maximum is 63.57 for Fama-French Beta.   

>s coefficient 

i). The average s coefficient across all stocks is 0.58   
ii). The minimum is -26.69, the 25th percentile is -0.07, the median is 0.39, the 75th percentile is 1.03, and the maximum is 21.23 for s coefficient.   

>h coefficient  

i). The average h coefficient across all stocks is -0.03    
ii). The minimum is -23.10, the 25th percentile is -0.48, the median is 0.02, the 75th percentile is 0.55, and the maximum is 10.34 for h coefficient. 

**There are some outliers since those maximum and minimum are far away those IQR boundaries.**

In [0]:
from scipy.stats.mstats import winsorize
# Convert variables
# Using winsorise to convert outliers
for col in fama_french.columns:
  fama_french[col] = winsorize(fama_french[col], limits=[0.06,0.07], inplace=True)
  #stats.winsorize(limits = [lower,upper])

In [0]:
fama_french.describe()

,FF-Beta,FF-Size,FF-Value
count,1364.000000,1364.000000,1364.000000
mean,1.043025,0.536846,-0.005222
std,0.539098,0.820563,0.759214
min,0.148600,-0.672656,-1.582187
25%,0.642078,-0.070789,-0.475491
50%,1.027332,0.394918,0.017082
75%,1.407668,1.026995,0.551794
max,2.030379,2.254943,1.193936


**Summarize**:     

>Fama-French Beta  

i). The average Fama-French Beta across all stocks is 1.04  
ii). The minimum is 0.15, the 25th percentile is 0.64, the median is 1.03, the 75th percentile is 1.41, and the maximum is 2.03 for Fama-French Beta,which now is the normal range for Fama-French Beta (0,2)   

>s coefficient 

i). The average s coefficient across all stocks is 0.54   
ii). The minimum is -0.67, the 25th percentile is -0.07, the median is 0.39, the 75th percentile is 1.03, and the maximum is 2.25 for s coefficient.   

>h coefficient  

i). The average h coefficient across all stocks is -0.01    
ii). The minimum is -1.58, the 25th percentile is -0.48, the median is 0.02, the 75th percentile is 0.55, and the maximum is 1.19 for h coefficient. 

**The ranges are more reasonable after winsorize the outliners**

### Export Data File

In [0]:
ticker = df4['Ticker'].value_counts()
ticker = ticker.sort_index(ascending=True).keys().to_frame(name='Ticker')
tmp = pd.concat([ticker,capm],axis=1)
tmp.head()

,Ticker,CAPM_Beta
Ticker,,
A,A,1.249362
AAL,AAL,1.803727
AAON,AAON,0.950044
AAP,AAP,1.006860
AAPL,AAPL,1.172580


In [0]:
ticker2 = ticker.sort_index(ascending=True)
ticker2
fama_french.set_index(ticker2['Ticker'],inplace = True)
fama_french.head()

,FF-Beta,FF-Size,FF-Value
Ticker,,,
A,1.397810,-0.540208,-0.310123
AAL,1.798855,-0.025965,0.686844
AAON,0.928965,0.078249,0.020157
AAP,0.875947,0.453546,0.627367
AAPL,1.296143,-0.416180,-0.776466


In [0]:
results = pd.concat([tmp, fama_french], axis=1)

In [0]:
results.index.name = None

In [0]:
results.reset_index(drop=True, inplace=True)
results

,Ticker,CAPM_Beta,FF-Beta,FF-Size,FF-Value
0,A,1.249362,1.397810,-0.540208,-0.310123
1,AAL,1.803727,1.798855,-0.025965,0.686844
2,AAON,0.950044,0.928965,0.078249,0.020157
3,AAP,1.006860,0.875947,0.453546,0.627367
4,AAPL,1.172580,1.296143,-0.416180,-0.776466
...,...,...,...,...,...
1359,ZLAB,1.348280,1.189334,0.738378,-1.551609
1360,ZM,0.164903,0.390590,-0.212080,-1.582187
1361,ZNGA,0.504836,0.159675,1.369960,-1.042994
1362,ZS,1.101106,1.595808,-0.672656,-1.582187


In [0]:
results.isna().any()

Ticker       False
CAPM_Beta    False
FF-Beta      False
FF-Size      False
FF-Value     False
dtype: bool

In [0]:
results.to_excel("Assign4-Output.xlsx",index=False,header=True)

## Bonus

### Import Data

In [0]:
from google.colab import files
uploaded = files.upload()

Saving BA870 Project Data 2020.xlsx to BA870 Project Data 2020.xlsx


In [0]:
assignoutput = pd.DataFrame(pd.read_excel('Assign4-Output.xlsx'))
projectdata = pd.DataFrame(pd.read_excel('BA870 Project Data 2020.xlsx'))
projectdata.drop(['Ticker'],axis=1,inplace=True)

### Combine Data and Data Cleaning

In [0]:
combinedfile = pd.merge(assignoutput, projectdata, left_on= 'Ticker', right_on= 'tic').drop('tic',axis=1)
combinedfile.to_csv('Assign4-Combine.csv', index=False, header=True)

In [0]:
bonus = pd.DataFrame(pd.read_csv('Assign4-Combine.csv')[['YTD Ret', 'CAPM_Beta', 'FF-Beta', 'FF-Size', 'FF-Value']])
bonus

,YTD Ret,CAPM_Beta,FF-Beta,FF-Size,FF-Value
0,-0.1745,1.249362,1.397810,-0.540208,-0.310123
1,-0.6726,1.803727,1.798855,-0.025965,0.686844
2,-0.0911,0.950044,0.928965,0.078249,0.020157
3,-0.4715,1.006860,0.875947,0.453546,0.627367
4,-0.1779,1.172580,1.296143,-0.416180,-0.776466
...,...,...,...,...,...
1325,0.2395,1.348280,1.189334,0.738378,-1.551609
1326,0.8842,0.164903,0.390590,-0.212080,-1.582187
1327,0.0850,0.504836,0.159675,1.369960,-1.042994
1328,0.3634,1.101106,1.595808,-0.672656,-1.582187


In [0]:
#check any NA in each variable in bonus dataset
bonus.isna().any()

YTD Ret      False
CAPM_Beta    False
FF-Beta      False
FF-Size      False
FF-Value     False
dtype: bool

### OLS Regression

In [0]:
import statsmodels.api as sm
Y = bonus.iloc[:, 0]
X = sm.add_constant(bonus.iloc[:, 1 :])
model = sm.OLS(Y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                YTD Ret   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.138
Method:                 Least Squares   F-statistic:                     54.12
Date:                Sun, 12 Apr 2020   Prob (F-statistic):           2.71e-42
Time:                        19:25:56   Log-Likelihood:                 129.72
No. Observations:                1330   AIC:                            -249.4
Df Residuals:                    1325   BIC:                            -223.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2182      0.014    -15.693      0.000      -0.245      -0.191
CAPM_Beta     -0.0125      0.037     -0.338      0.736      -0.085       0.060
FF-Beta       -0.0708      0.038     -1.876      0.061      -0.145       0.003
FF-Size       -0.0216      0.012     -1.757      0.079      -0.046       0.003
FF-Value      -0.0976      0.008    -11.983      0.000      -0.114      -0.082
==============================================================================
Omnibus:                      533.124   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7110.509
Skew:                           1.483   Prob(JB):                         0.00
Kurtosis:                      13.932   Cond. No.                         18.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Interpretation

The estimates of CAPM_Beta, FF-Beta, FF-Size, FF-Value are -0.0125, -0.0708, -0.0216, and -0.0976 respectively. Those p-values for those Betas are 0.736, 0.061, 0.079, and 0 which means they are not statistically significant in the multiple linear regression except `FF-Value`.   

The R-Square is 0.140, which means 14.0% of 2020 YTD Ret is explained by those Betas. In our opinion, though our model explained a part of variability of 2020 YTD Ret, our multiple regression model is still not a very good fit. 

Also the Adj.R-squared is only 0.138, a more conservative estimate by penalizing for the number of
independent variables in the model. Thus Beta, S and/or h are not explain variable variation in 2020 YTD Ret very well.

#### Check Multicollinear

In [0]:
#correlation matrix
corr_matrix = bonus.corr().abs()
corr_matrix

,YTD Ret,CAPM_Beta,FF-Beta,FF-Size,FF-Value
YTD Ret,1.000000,0.217250,0.190986,0.069194,0.314551
CAPM_Beta,0.217250,1.000000,0.883581,0.337122,0.063696
FF-Beta,0.190986,0.883581,1.000000,0.037949,0.020724
FF-Size,0.069194,0.337122,0.037949,1.000000,0.030830
FF-Value,0.314551,0.063696,0.020724,0.030830,1.000000


According to the correlation matrix,  the CAPM_Beta and FF-Beta are highly correlated (0.88), so we need to remove `CAPM_BETA` from model.

#### New OLS Regression

In [0]:
Y1 = bonus.iloc[:, 0]
X1 = sm.add_constant(bonus.iloc[:, 2 :])
model1 = sm.OLS(Y1,X1).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                YTD Ret   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.138
Method:                 Least Squares   F-statistic:                     72.17
Date:                Sun, 12 Apr 2020   Prob (F-statistic):           3.10e-43
Time:                        19:25:56   Log-Likelihood:                 129.66
No. Observations:                1330   AIC:                            -251.3
Df Residuals:                    1326   BIC:                            -230.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2185      0.014    -15.751      0.000      -0.246      -0.191
FF-Beta       -0.0829      0.011     -7.367      0.000      -0.105      -0.061
FF-Size       -0.0250      0.007     -3.373      0.001      -0.040      -0.010
FF-Value      -0.0982      0.008    -12.297      0.000      -0.114      -0.083
==============================================================================
Omnibus:                      529.563   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6994.460
Skew:                           1.473   Prob(JB):                         0.00
Kurtosis:                      13.842   Cond. No.                         4.62
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Reinterpretation
Now, every variabel is statistically significant in the multiple linear regression since all of their p-values are less than 0.05.   
The R-Square is still **0.140**, which means **14.0%** of 2020 YTD Ret is explained by those Betas, so it's not a good fit of multiple linear regression. And the Adj.R-squared is still only 0.138 which adjusts for the number of independent variables in the model, considered the
more conservative estimate. **Thus Beta, S and/or h are not explain variable variation in 2020 YTD Ret very well.**